# Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- title: Unit 2.4a (Using Programs with Data, SQLAlchemy) Notes
- permalink: /2-4alessonnotes/
- categories: []

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 

## Imports and Flask Objects

> Defines and key object creations

- Comment on where you have observed these working?

1. **Flask app object**: Our Flask servers that we created our databases on obviously used Flask previously. It often ends up in conjunction with...

2. **SQLAlchemy object**: This is always associated with our recent database creations. It is used both to work with others' data and create our own.

In [38]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class

- **`class User` purpose**: Abstracts a user into a specific, named object and creates variables for each of its attributes and traits.

- **`db.Model` inheritance**: Takes attributes and aspects of `db.Model` when creating the Users class.

- **`__init__` method**: Acts as the constructor for the class, which defines each of the object's attributes.

- **`@property` (getter), `@[column].setter` (setter)**: Getter and setter functions, allows for reading an attribute respectively.

- **Additional methods**: `delete` and `update` functions that I use later.

In [39]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts email from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows name to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter versus stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, to avoid unfriendly outcomes
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob should be have verification for type date
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password="", dob=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        else:
            self.dob = dob
        db.session.add(self)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work.

1. **Create All Tables from `db` Object**: The `create_all()` function creates the SQLite table, allowing for the user objects to be added into it.

2. **`User` Object Constructors**: The user object is created differently for each user variable below. The constructors create a user object with the attributes defined by the arguments provided to the `User` class.

3. **`Try` / `Except`**: Using the `try` condition means that, if the user creation fails, there is a failsafe to the error in the `except` case that doesn't completely destroy the process. It also acts as a validation process.

In [40]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Created new uid toby
Created new uid niko
Created new uid lex
Created new uid whit
Created new uid indi
Created new uid raven


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following

1. **`User.query.filter_by`**: Finds a specific user and its data based on the given UID, which will then have its password used as a checker.

2. **`user.password`**: Used to varify the password input after getting the UID in question.

In [41]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following

1. **`user.find_by_uid()` and `try`/`except`**: The `find_by_uid` function is used to define the `user` function as the user with the given `uid` variable. However, the `find_by_uid` process will fail if a user with the inputted `uid` doesn't exist. As a result, the `print` function under the `try` condition will have an error when trying to read the `user` variable if it was never defined, and the `except` condition will occur, using `pass` to transition in to the user creation process.

2. **`user = User(...)`**: The `user` variable is defined as a `User` object with the name, user ID and password inputted above. The placement of this definition/initialization process is important because of the `user.dob` definition (described below).

3. **`user.dob` and `try`/`except`**: After defining `user` as the `User` object, the program attempts to create the user's date of birth based on the inputted year, month and day by directly adding the attribute using `user.dob`. If the date of birth was inputted incorrectly, the `.strptime` function will fail and the `except` condition will be triggered, setting a default user date of birth as the current date.

4. **`user.create()` and `try`/`except`**: The `user.create()` function adds the user object to the database. This is put within a `try` condition because, if for some reason this fails, the `except` function will be used to cancel the process and state that an unknown error occurred.

In [42]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'name': 'AJ Ruiz', 'uid': 'ajbal', 'dob': '04-16-2006', 'age': 16}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following

1. `User.query.all`: This represents the full table of User data, which is then defined as the local `table` variable which is iterated through to create the JSON.

2. `json_ready` assignment: The `json_ready` variable represents the correctly-formatted JSON data formed by the procedure beside it. It is then returned with the function.

In [43]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # each user adds user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': 'AJ Ruiz', 'uid': 'ajbal', 'dob': '04-16-2006', 'age': 16}]

# Hacks

Below is evidence of my completion of each of the hacks.

## 2.4 College Board Quiz Questions

Here are my results on the 2.4 College Board quiz.

<figure>
  <img src="{{site.baseurl}}/images/Screen Shot 2023-03-15 at 10.02.05 PM.png" alt="final table" width="800"/>
  <figcaption>I got 6/6.</figcaption>
</figure>

The questions were all fairly self-explanatory. However, in many cases, it required reading through the problem very thoroughly.

The main reason a lot of the questions weren't challenging was that certain aspects of the problem were clearly irrelevant to the prompt given, so if I saw an answer providing the use of that aspect in some way, chances were that it was incorrect. Question 3 had most of the "contents" column as completely useless information, and Question 4 had the "Sort by Subject" procedure that was clearly irrelevant to the given prompt.

The only question I had to take some time to think about was question 2, because each of the responses were so detailed and specific to the information provided that I had to focus pretty hard on each of them.

## Update Function

In the code cell below, I created a function to update user data in the database.

In [51]:
# Inputs, Try/Except, and SQLAlchemy work together AGAIN to update the user
def update():
    # first require uid input
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    if user != None:
        print('Found user with user ID "' + uid + '".')
        pass
    else:
        print("User not found.")
        return
    
    # validate user with password
    password = input("Enter your password to verify your identity.")
    if check_credentials(uid, password):
        print("Password varified. You will now input updated user information.")
        pass
    else:
        print("Invalid password.")
        return
    
    # create new User attributes
    newname = input("Enter your name.")
    newuid = input("Input a user ID. It can be a new one if you'd like.")
    check = find_by_uid(newuid)
    if (check != "None") and (check != uid):
        print(f"The user ID {newuid} is already being used by another user.")
        return
    newps = input("Input a password. It can be a new one if you'd like.")
    newdob = input("Enter your date of birth in 'YYYY-MM-DD' format.")
    try:
        dobconv = datetime.strptime(newdob, '%Y-%m-%d').date()
    except ValueError:
        dobconv = datetime.today()
        print(f"Invalid date {newdob}: incorrect YYYY-mm-dd format. Date defaulted to {dobconv}")
           
    # update the user in the database
    with app.app_context():
        try:
            object = user.update(name=newname, uid=newuid, password=newps, dob=dobconv)
            print("Updated user:\n", object.read())
        except:  # error raised if object not created
            print("Unknown error.")
        
update()

Found user with user ID "kkcbal".
Password varified. You will now input updated user information.
Updated user:
 {'id': 7, 'name': 'AJ Ruiz', 'uid': 'kkcbal', 'dob': '05-18-2006', 'age': 16}


## Delete Function

Below is a function that allows for the deletion of a certain user if they verify their user ID and password.

In [57]:
def delete():
    # first require uid input
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    if user != None:
        print('Found user with user ID "' + uid + '".')
        pass
    else:
        print("User not found.")
        return
    
    # validate user with password
    password = input("Enter your password to verify your identity.")
    if check_credentials(uid, password):
        print('Password varified. User with ID "' + uid + '" will be deleted...')
        pass
    else:
        print("Invalid password.")
        return
    
    with app.app_context():
        try:
            rslt = user.delete()
            print("User has been deleted successfully.")
        except:
            print("There was an unknown error deleting the given user.")

delete()

Found user with user ID "lex".
Password varified. User with ID "lex" will be deleted...
User has been deleted successfully.


I decided to delete the user Alexander Graham Bell for an example. After running all of the above processes, the SQLite table looks like this:

<figure>
  <img src="{{site.baseurl}}/images/Screen Shot 2023-03-15 at 9.08.01 PM.png" alt="final table" width="650"/>
  <figcaption>ID 3 (Alexander Graham Bell) has been deleted. ID 7 (AJ Ruiz) has been updated since his initial creation (see the output of the Create function).</figcaption>
</figure>